# Fake Page View generator

This notebook generates page views based on a three-hour dataset.

In [1]:
%%bash
# Install the required Python 3 dependencies
python3 -m pip install kafka-python  # type: ignore

In [2]:
# Extract archive of click data
import zipfile
import os

ARCHIVEF = "20180113-6-9-with-nulls.json.zip"
CLICKSF = "20180113-6-9-with-nulls.json"

if (not os.path.isfile(CLICKSF)):
    with zipfile.ZipFile(ARCHIVEF,"r") as zip_ref:
        zip_ref.extractall(".")

In [4]:
# Send the ratings to the endpoint in order to test your implementation of the endpoint.
# SENDMODE = "http"

# Send the ratings straight to Kafka in order to simulate a working endpoint.
# This allows you to test cleaning code without a working endpoint. 
SENDMODE = "kafka"


In [6]:
# Send click data
import json
import time
from datetime import datetime
import itertools

import requests
from kafka import KafkaProducer
from kafka.errors import KafkaError
from IPython.display import clear_output

producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

orig_start = 0
replay_start = time.time() * 1000

counter = 0
i = 0

print("Simulating website traffic...")

with open(CLICKSF, 'r') as f:
    first = True
    start_ts = 0
    for line in itertools.islice(f, 100):
        click = json.loads(line)

        orig_event = click["ts_ingest"] or 0
        
        if orig_event:
            curr_ts = time.time() * 1000

            if first:
                first = False
                orig_start = click["ts_ingest"]

            # difference = (orig_event - orig_start) - (curr_ts - replay_start)

            # if difference > 0:
            #     print("sleeping {0:.2f} seconds".format(difference/1000.0))
            #     time.sleep(difference/1000.0)
            
        click['ts_ingest'] = round((orig_event - orig_start) + replay_start)

        if SENDMODE == "kafka":            
            future = producer.send('clicks', click)
            try:
                record_metadata = future.get(timeout=10)
            except KafkaError:
                log.exception()
                pass
        else:
            #print(f"sending {line}")
            requests.post("http://localhost:5000/clicks", data=json.dumps(click).encode("utf-8"), headers={'Content-Type':'application/json'})
        
        print("✓ {}".format(datetime.fromtimestamp(orig_event/1000.0)))
        if i > 1000:
            clear_output()
            i = 0
        i = i+1


Simulating website traffic...
✓ 2018-01-13 05:04:04.345000
✓ 2018-01-13 05:04:04.349000
✓ 2018-01-13 05:04:04.359000
✓ 2018-01-13 05:04:04.409000
✓ 1970-01-01 00:00:00
✓ 2018-01-13 05:04:04.416000
✓ 2018-01-13 05:04:04.469000
✓ 2018-01-13 05:04:04.487000
✓ 2018-01-13 05:04:04.598000
✓ 2018-01-13 05:04:04.616000
✓ 2018-01-13 05:04:04.625000
✓ 2018-01-13 05:04:04.652000
✓ 2018-01-13 05:04:04.687000
✓ 2018-01-13 05:04:04.689000
✓ 1970-01-01 00:00:00
✓ 2018-01-13 05:04:04.703000
✓ 2018-01-13 05:04:04.758000
✓ 2018-01-13 05:04:04.790000
✓ 2018-01-13 05:04:04.842000
✓ 2018-01-13 05:04:04.927000
✓ 2018-01-13 05:04:04.931000
✓ 2018-01-13 05:04:04.958000
✓ 2018-01-13 05:04:04.979000
✓ 2018-01-13 05:04:05.089000
✓ 2018-01-13 05:04:05.099000
✓ 2018-01-13 05:04:05.207000
✓ 2018-01-13 05:04:05.248000
✓ 2018-01-13 05:04:05.300000
✓ 2018-01-13 05:04:05.305000
✓ 2018-01-13 05:04:05.315000
✓ 1970-01-01 00:00:00
✓ 2018-01-13 05:04:05.342000
✓ 2018-01-13 05:04:05.362000
✓ 2018-01-13 05:04:05.386000
✓ 201